# Problem Set 2: Nonlinear vs Linearized Least Squares Regression

AI 211 | 2nd Sem AY 2023-2024

Students:  
Joshua Cantor  
Michael Spencer Quinto


In [3]:
import numpy as np
import matplotlib.pyplot as plt

from numpy import ndarray

# Nonlinear vs Linearized Least Squares Regression

The Michaelis-Menten equation describes the chemical kinetics of enzyme reactions. According to this equation, if $v_0$ is the initial velocity, $v_{max}$, the maximum velocity, $K_m$ is the Michaelis constant for the reaction and $C$ is the substrate concentration, then

$v_0 = \frac{v_{max}}{1 + \frac{K_m}{C}}.$

In a typical experiment, $v_0$ is measured as $C$ is varied, and then $v_{max}$ and $K_m$ are determined from the resulting data using curve fitting.

| C    | v_0   |
| ---- | ----- |
| 2.5  | 0.024 |
| 5.0  | 0.036 |
| 10.0 | 0.053 |
| 15.0 | 0.060 |
| 20.0 | 0.064 |
